# Relaciones

In [60]:
#conexiones a aura
from neo4j import GraphDatabase
uri = "neo4j+ssc://1e48c053.databases.neo4j.io"
auth= ("neo4j", "R_ro7G4XS1ixvmtnxHX5_Kb0pXxZ_Lohg91TPLoaHZQ")


In [2]:
with GraphDatabase.driver(uri, auth=auth) as driver:
    driver.verify_connectivity()

In [20]:
#un solo label
def crear_nodo(uri, auth, label, propiedades):
    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        resultado = session.run(f"CREATE (n:{label} $props) RETURN n", props=propiedades)
        nodo = resultado.single()[0]
    driver.close()
    return nodo

In [26]:
#2 o más labels
def crear_nodo_con_labels(uri, auth, labels, propiedades):
    # Conecta a la base de datos
    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        labels_str = ":" + ":".join(labels)
        query = f"CREATE (n{labels_str} $props) RETURN n"
        resultado = session.run(query, props=propiedades)
        nodo = resultado.single()[0]
    driver.close()
    return nodo

In [22]:
def consultar_nodos(uri, auth, label=None, filtros=None, solo_uno=False):
    """
    :param label: Label del nodo (opcional). Si no se especifica, buscará nodos sin filtrar por label.
    :param filtros: Diccionario con propiedades para filtrar. Ej: {"nombre": "Juan", "edad": 30}
    :param solo_uno: Si es True, retorna solo el primer nodo que coincida con los filtros. 
                     Si es False, retorna todos los nodos que coincidan
    """
    
    driver = GraphDatabase.driver(uri, auth=auth)
    
    with driver.session() as session:
        # Construimos la parte inicial del query: MATCH (n:Label)
        query = "MATCH (n"
        if label:
            query += f":{label}"
        query += ")"
        
        # Si se especifican filtros, creamos condiciones en el WHERE
        if filtros:
            condiciones = []
            for key in filtros.keys():
                # Genera algo como "n.nombre = $props.nombre"
                condiciones.append(f"n.{key} = $props.{key}")
            query += " WHERE " + " AND ".join(condiciones)
        
        # Decidimos si queremos un único nodo o todos
        if solo_uno:
            query += " RETURN n LIMIT 1"
        else:
            query += " RETURN n"
        
        # Ejecutamos el query, pasando las propiedades en 'props'
        resultado = session.run(query, props=filtros)
        
        if solo_uno:
            # Retornamos el primer nodo si existe
            registro = resultado.single()
            if registro:
                return registro["n"]
            else:
                return None
        else:
            # Retornamos una lista de todos los nodos encontrados
            nodos = []
            for registro in resultado:
                nodos.append(registro["n"])
            return nodos
    
    driver.close()

In [34]:
def agregar_propiedades_nodo_por_filtro(uri, auth, label, filtros, propiedades):
    """
    Agrega (o sobrescribe) propiedades en un único nodo que cumpla con el filtro.
    Si el filtro coincide con varios, solo se actualiza el primero.
    """
    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        # Construir la cláusula WHERE directamente si se proporcionan filtros
        where_clause = ""
        if filtros:
            condiciones = [f"n.{key} = $filtros.{key}" for key in filtros]
            where_clause = "WHERE " + " AND ".join(condiciones)
        
        query = f"""
        MATCH (n:{label})
        {where_clause}
        SET n += $props
        RETURN n
        LIMIT 1
        """
        resultado = session.run(query, filtros=filtros, props=propiedades)
        registro = resultado.single()
        driver.close()
        return registro["n"] if registro else None


In [35]:
def agregar_propiedades_nodos_por_filtro(uri, auth, label, filtros, propiedades):
    """
    Agrega (o sobrescribe) propiedades a todos los nodos que cumplan con el filtro.
    """
    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        where_clause = ""
        if filtros:
            condiciones = [f"n.{key} = $filtros.{key}" for key in filtros]
            where_clause = "WHERE " + " AND ".join(condiciones)
        
        query = f"""
        MATCH (n:{label})
        {where_clause}
        SET n += $props
        RETURN n
        """
        resultado = session.run(query, filtros=filtros, props=propiedades)
        nodos = [registro["n"] for registro in resultado]
        driver.close()
        return nodos

In [36]:
def actualizar_propiedades_nodo_por_filtro(uri, auth, label, filtros, propiedades):
    """
    Actualiza (o agrega) propiedades en un único nodo que cumpla con el filtro.
    """
    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        where_clause = ""
        if filtros:
            condiciones = [f"n.{key} = $filtros.{key}" for key in filtros]
            where_clause = "WHERE " + " AND ".join(condiciones)
        
        query = f"""
        MATCH (n:{label})
        {where_clause}
        SET n += $props
        RETURN n
        LIMIT 1
        """
        resultado = session.run(query, filtros=filtros, props=propiedades)
        registro = resultado.single()
        driver.close()
        return registro["n"] if registro else None

In [38]:
def actualizar_propiedades_nodos_por_filtro(uri, auth, label, filtros, propiedades):
    """
    Actualiza (o agrega) propiedades en todos los nodos que cumplan con el filtro.
    """
    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        where_clause = ""
        if filtros:
            condiciones = [f"n.{key} = $filtros.{key}" for key in filtros]
            where_clause = "WHERE " + " AND ".join(condiciones)
        
        query = f"""
        MATCH (n:{label})
        {where_clause}
        SET n += $props
        RETURN n
        """
        resultado = session.run(query, filtros=filtros, props=propiedades)
        nodos = [registro["n"] for registro in resultado]
        driver.close()
        return nodos

In [39]:
def eliminar_propiedades_nodo_por_filtro(uri, auth, label, filtros, lista_propiedades):
    """
    Elimina propiedades específicas de un único nodo que cumpla con el filtro.
    Si el filtro coincide con varios, solo se elimina de uno.
    """
    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        where_clause = ""
        if filtros:
            condiciones = [f"n.{key} = $filtros.{key}" for key in filtros]
            where_clause = "WHERE " + " AND ".join(condiciones)
        
        query = f"""
        MATCH (n:{label})
        {where_clause}
        FOREACH (key IN $keys | REMOVE n[key])
        RETURN n
        LIMIT 1
        """
        resultado = session.run(query, filtros=filtros, keys=lista_propiedades)
        registro = resultado.single()
        driver.close()
        return registro["n"] if registro else None


In [40]:
def eliminar_propiedades_nodos_por_filtro(uri, auth, label, filtros, lista_propiedades):
    """
    Elimina propiedades específicas de todos los nodos que cumplan con el filtro.
    """
    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        where_clause = ""
        if filtros:
            condiciones = [f"n.{key} = $filtros.{key}" for key in filtros]
            where_clause = "WHERE " + " AND ".join(condiciones)
        
        query = f"""
        MATCH (n:{label})
        {where_clause}
        FOREACH (key IN $keys | REMOVE n[key])
        RETURN n
        """
        resultado = session.run(query, filtros=filtros, keys=lista_propiedades)
        nodos = [registro["n"] for registro in resultado]
        driver.close()
        return nodos

In [47]:
def crear_relacion_con_propiedades_por_filtro(
    uri, auth,
    label_a, filtros_a,
    label_b, filtros_b,
    tipo_relacion,
    propiedades_relacion
):
    """
    Crea una relación con un tipo y propiedades entre dos nodos existentes 
    buscados por filtros (en lugar de usar IDs internos).

    :param label_a: Label del primer nodo (p.ej. "Persona")
    :param filtros_a: Diccionario de propiedades para filtrar el primer nodo (p.ej. {"uuid": "abc123"})
    :param label_b: Label del segundo nodo (p.ej. "Persona")
    :param filtros_b: Diccionario de propiedades para filtrar el segundo nodo (p.ej. {"uuid": "xyz789"})
    :param tipo_relacion: Tipo de la relación (p.ej. "AMIGO_DE", "CONOCE_A", etc.)
    :param propiedades_relacion: Diccionario con **al menos 3** propiedades para la relación
    :return: El objeto de relación creado (neo4j.graph.Relationship) o None si no se encontró ningún par de nodos
    """

    # Verificamos que la relación tenga al menos 3 propiedades
    if len(propiedades_relacion) < 3:
        raise ValueError("Se requieren al menos 3 propiedades para la relación.")

    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        # Construimos la cláusula WHERE para cada nodo en función de los filtros
        where_clause_a = ""
        if filtros_a:
            condiciones_a = [f"a.{k} = $filtrosA.{k}" for k in filtros_a]
            where_clause_a = " AND ".join(condiciones_a)
        
        where_clause_b = ""
        if filtros_b:
            condiciones_b = [f"b.{k} = $filtrosB.{k}" for k in filtros_b]
            where_clause_b = " AND ".join(condiciones_b)
        
        # Unimos las condiciones en un WHERE final (si existen)
        if where_clause_a and where_clause_b:
            final_where = f"WHERE {where_clause_a} AND {where_clause_b}"
        elif where_clause_a:
            final_where = f"WHERE {where_clause_a}"
        elif where_clause_b:
            final_where = f"WHERE {where_clause_b}"
        else:
            final_where = ""

        # Creamos el query
        query = f"""
        MATCH (a:{label_a}), (b:{label_b})
        {final_where}
        CREATE (a)-[r:{tipo_relacion} $relProps]->(b)
        RETURN r
        """

        # Ejecutamos el query
        result = session.run(
            query,
            filtrosA=filtros_a,
            filtrosB=filtros_b,
            relProps=propiedades_relacion
        )
        
        # Obtenemos la primera relación creada (o None si no se creó ninguna)
        record = result.single()
        relacion_creada = record["r"] if record else None

    driver.close()
    return relacion_creada

In [69]:
def eliminar_un_nodo_por_filtro(uri, auth, label, filtros):
    """
    Elimina un único nodo que cumpla con el filtro, junto con sus relaciones.
    Retorna True si se eliminó el nodo, False si no se encontró ninguno.
    """
    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        # Construcción dinámica del WHERE
        where_clause = ""
        if filtros:
            condiciones = [f"n.{key} = $filtros.{key}" for key in filtros]
            where_clause = "WHERE " + " AND ".join(condiciones)
        
        query = f"""
        MATCH (n:{label})
        {where_clause}
        WITH n LIMIT 1
        DETACH DELETE n
        RETURN count(n) AS nodosEliminados
        """
        resultado = session.run(query, filtros=filtros)
        record = resultado.single()
        nodos_eliminados = record["nodosEliminados"] if record else 0
        
    driver.close()
    return nodos_eliminados > 0


In [49]:
def eliminar_multiples_nodos_por_labels_y_filtros(uri, auth, labels, filtros):
    """
    Elimina todos los nodos que tengan todos los labels en `labels`
    y que cumplan con las propiedades de `filtros`. También elimina 
    sus relaciones (DETACH). Retorna la cantidad de nodos eliminados.
    
    :param labels: Lista de labels (p.ej. ["Persona", "Empleado"]).
                   El nodo debe tener todas esas labels.
    :param filtros: Diccionario de propiedades para filtrar (p.ej. {"edad": 30}).
    """
    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        # Construir la parte de los labels, p.ej. :Persona:Empleado
        labels_str = ":" + ":".join(labels)  # ["Persona", "Empleado"] -> ":Persona:Empleado"

        # Construir la cláusula WHERE con las propiedades del diccionario
        where_clause = ""
        if filtros:
            condiciones = [f"n.{key} = $filtros.{key}" for key in filtros]
            where_clause = "WHERE " + " AND ".join(condiciones)
        
        query = f"""
        MATCH (n{labels_str})
        {where_clause}
        DETACH DELETE n
        RETURN count(n) AS nodosEliminados
        """
        
        resultado = session.run(query, filtros=filtros)
        record = resultado.single()
        nodos_eliminados = record["nodosEliminados"] if record else 0

    driver.close()
    return nodos_eliminados

In [51]:
def eliminar_una_relacion_por_filtro(
    uri, auth,
    label_a, filtros_a,
    tipo_relacion,
    label_b, filtros_b
):
    """
    Elimina una sola relación (la primera que coincida) entre dos nodos que cumplan los filtros.
    Retorna True si se eliminó alguna relación, False si no se encontró ninguna.
    
    - label_a, filtros_a: para identificar el nodo de origen
    - tipo_relacion: el tipo de la relación (ej: "AMIGO_DE")
    - label_b, filtros_b: para identificar el nodo de destino
    """
    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        where_clause_a = ""
        if filtros_a:
            condiciones_a = [f"a.{k} = $filtrosA.{k}" for k in filtros_a]
            where_clause_a = " AND ".join(condiciones_a)
        
        where_clause_b = ""
        if filtros_b:
            condiciones_b = [f"b.{k} = $filtrosB.{k}" for k in filtros_b]
            where_clause_b = " AND ".join(condiciones_b)

        # Unir condiciones en un único WHERE (si existen)
        if where_clause_a and where_clause_b:
            final_where = f"WHERE {where_clause_a} AND {where_clause_b}"
        elif where_clause_a:
            final_where = f"WHERE {where_clause_a}"
        elif where_clause_b:
            final_where = f"WHERE {where_clause_b}"
        else:
            final_where = ""
        
        query = f"""
        MATCH (a:{label_a})-[r:{tipo_relacion}]->(b:{label_b})
        {final_where}
        WITH r LIMIT 1
        DELETE r
        RETURN count(r) AS relacionesEliminadas
        """
        resultado = session.run(
            query,
            filtrosA=filtros_a,
            filtrosB=filtros_b
        )
        record = resultado.single()
        relaciones_eliminadas = record["relacionesEliminadas"] if record else 0

    driver.close()
    return relaciones_eliminadas > 0

In [54]:
def eliminar_multiples_relaciones_por_filtro(
    uri, auth,
    label_a, filtros_a,
    tipo_relacion,
    label_b, filtros_b
):
    """
    Elimina todas las relaciones que coincidan entre nodos que cumplan los filtros.
    Retorna la cantidad de relaciones eliminadas.
    """
    driver = GraphDatabase.driver(uri, auth=auth)
    with driver.session() as session:
        where_clause_a = ""
        if filtros_a:
            condiciones_a = [f"a.{k} = $filtrosA.{k}" for k in filtros_a]
            where_clause_a = " AND ".join(condiciones_a)
        
        where_clause_b = ""
        if filtros_b:
            condiciones_b = [f"b.{k} = $filtrosB.{k}" for k in filtros_b]
            where_clause_b = " AND ".join(condiciones_b)

        if where_clause_a and where_clause_b:
            final_where = f"WHERE {where_clause_a} AND {where_clause_b}"
        elif where_clause_a:
            final_where = f"WHERE {where_clause_a}"
        elif where_clause_b:
            final_where = f"WHERE {where_clause_b}"
        else:
            final_where = ""
        
        query = f"""
        MATCH (a:{label_a})-[r:{tipo_relacion}]->(b:{label_b})
        {final_where}
        DELETE r
        RETURN count(r) AS relacionesEliminadas
        """
        resultado = session.run(
            query,
            filtrosA=filtros_a,
            filtrosB=filtros_b
        )
        record = resultado.single()
        relaciones_eliminadas = record["relacionesEliminadas"] if record else 0

    driver.close()
    return relaciones_eliminadas


In [52]:
def primer_menu():
    print("¿Qué desea realizar?")
    print("1. Crear un nodo")
    print("2. Vizualizar nodos")
    print("3. Operaciones con nodos")
    print("4. Crear relaciones")
    print("5. Gestión de relaciones")
    print("6. Eliminar nodos")
    print("7. Eliminar relaciones")
    print("8. Salir")

In [84]:
def menu_nodos():
    print("¿Qué desesa realizar?")
    print("1. Creación de nodos con 1 label")
    print("2. Creación de nodos con 2 o más lables")
    res2 = input("Elija una opción (solo colocar el número de opción): ")

    if res2 == "1":
        print("Escriba el label del nodo que va a ingresar: ")
        label = input("")
        preg1 = input("¿Quiere agreagr propiedades? (Y/N) ")
        if preg1 == "Y":
            n = int(input("¿Cuántas propiedades quiere agregar? "))
            propiedades = {input("Propiedad: "): input("Valor: ") for i in range(n)}

        else: 
            propiedades = {}

        crear_nodo(uri, auth, label, propiedades)

    elif res2 == "2":
        n1 = int(input("¿Cuántos labels quiere ingresar? "))
        print("Escriba el nombre de los labeles del nodo que va a ingresar: ")
        labels = [input("Escriba el label ") for i in range(n1)]
        preg1 = input("¿Quiere agreagr propiedades? (Y/N) ")
        if preg1 == "Y":
            n = int(input("¿Cuántas propiedades quiere agregar? "))
            propiedades = {input("Propiedad: "): input("Valor: ") for i in range(n)}

        else: 
            propiedades = {}

        crear_nodo_con_labels(uri, auth, labels, propiedades)
        
    else:
        print("Escriba una opción válida")

In [97]:
def menu_consulta_nodos():
    res3 = input("¿Desea buscar por label? (Y/N) ").upper()
    res4 = input("¿Desea agregar filtros? (Y/N) ").upper()
    nodo = input("¿Desea consultar un nodo o todos los nodos (uno/varios)? ").lower()
    
    if res3 == "Y" and res4 == "Y" and nodo == "uno":
        label = input("Ingrese el label que quiere buscar: ")
        filtros = {input("Propiedad: "): input("Valor: ")}
        resultado = consultar_nodos(uri, auth, label, filtros, True)
        print("Resultado:", resultado)

    elif res3 == "Y" and res4 == "Y" and nodo == "varios":
        label = input("Ingrese el label que quiere buscar: ")
        filtros = {input("Propiedad: "): input("Valor: ")}
        resultado = consultar_nodos(uri, auth, label, filtros)
        print("Resultados:", resultado)

    elif res3 == "Y" and res4 == "N" and nodo == "uno":
        label = input("Ingrese el label que quiere buscar: ")
        resultado = consultar_nodos(uri, auth, label, filtros=None, solo_uno=True)
        print("Resultado:", resultado)

    elif res3 == "Y" and res4 == "N" and nodo == "varios":
        label = input("Ingrese el label que quiere buscar: ")
        resultado = consultar_nodos(uri, auth, label)
        print("Resultados:", resultado)

    elif res3 == "N" and res4 == "Y" and nodo == "uno":
        filtros = {input("Propiedad: "): input("Valor: ")}
        resultado = consultar_nodos(uri, auth, label=None, filtros=filtros, solo_uno=True)
        print("Resultado:", resultado)

    elif res3 == "N" and res4 == "Y" and nodo == "varios":
        filtros = {input("Propiedad: "): input("Valor: ")}
        resultado = consultar_nodos(uri, auth, label=None, filtros=filtros)
        print("Resultados:", resultado)

    elif res3 == "N" and res4 == "N" and nodo == "uno":
        resultado = consultar_nodos(uri, auth, label=None, filtros=None, solo_uno=True)
        print("Resultado:", resultado)

    elif res3 == "N" and res4 == "N" and nodo == "varios":
        resultado = consultar_nodos(uri, auth)
        print("Resultados:", resultado)
            

In [80]:
def menu_operaciones_nodos():
    print("¿Qué desea realizar? ")
    print("1. Agregar 1 o más propiedades a un nodo")
    print("2. Agregar 1 o más propiedades a múltiples nodos")
    print("3. Actualizar 1 o más propiedades de un nodo")
    print("4. Actualizar 1 o más propiedades de múltiples nodos")
    print("5. Eliminar 1 o más propiedades de un nodo")
    print("6. Eliminar 1 o más propiedades de múltiples nodos")

    res5 = input("Escoge un número: ")
    if res5 == "1":
        nodo = input("Escribe el nombre del nodo al que le quieres agregar propiedades: ")
        preg_res = input("¿Desea buscar por filtro? (Y/N) ")
        if preg_res == "Y":
            print("Agregue las propiedades y el valor para encontrar el nodo: ")
            filtros = {input("Propiedad: "): input("Valor: ")}

        else:
            filtros = {}
            
        n = int(input("¿Cuántas propiedades quiere agregar?"))
        propiedades = {input("Propiedad: "): input("Valor: ") for i in range(n)}
        
        agregar_propiedades_nodo_por_filtro(uri, auth, nodo, filtros, propiedades)

    elif res5 == "2":
        nodos = input("Escribe el nombre del nodo al que le quieres agregar propiedades: ")
        preg_res = input("¿Desea buscar por filtro? (Y/N) ")
        if preg_res == "Y":
            print("Agregue las propiedades y el valor para encontrar el nodo: ")
            filtros = {input("Propiedad: "): input("Valor: ")}
        else:
            filtros = {}
            
        n = int(input("¿Cuántas propiedades quiere agregar?"))
        propiedades = {input("Propiedad: "): input("Valor: ") for i in range(n)}
        
        agregar_propiedades_nodos_por_filtro(uri, auth, nodos, filtros, propiedades)

    elif res5 == "3":
        nodo = input("Escribe el nombre del nodo al que le quieres actualizar propiedades: ")
        preg_res = input("¿Desea buscar por filtro? (Y/N) ")
        if preg_res == "Y":
            print("Agregue las propiedades y el valor para encontrar el nodo: ")
            filtros = {input("Propiedad: "): input("Valor: ")}

        else:
            filtros = {}
            
        n = int(input("¿Cuántas propiedades quiere actualizar?"))
        propiedades = {input("Propiedad: "): input("Valor: ") for i in range(n)}

        actualizar_propiedades_nodo_por_filtro(uri, auth, nodo, filtros, propiedades)

    elif res5 == "4":
        nodo = input("Escribe el nombre del nodo al que le quieres actualizar propiedades: ")
        preg_res = input("¿Desea buscar por filtro? (Y/N) ")
        if preg_res == "Y":
            print("Agregue las propiedades y el valor para encontrar el nodo: ")
            filtros = {input("Propiedad: "): input("Valor: ")}

        else:
            filtros = {}
        
        n = int(input("¿Cuántas propiedades quiere actualizar?"))
        propiedades = {input("Propiedad: "): input("Valor: ") for i in range(n)}

        actualizar_propiedades_nodos_por_filtro(uri, auth, nodo, filtros, propiedades)

    elif res5 == "5":
        nodo = input("Escribe el nombre del nodo al que le quieres eliminar propiedades: ")
        preg_res = input("¿Desea buscar por filtro? (Y/N) ")
        if preg_res == "Y":
            print("Agregue las propiedades y el valor para encontrar el nodo: ")
            filtros = {input("Propiedad: "): input("Valor: ")}

        else:
            filtros = {}
            
        n = int(input("¿Cuántas propiedades quiere eliminar?"))
        propiedades = [input("Propiedad: ") for i in range(n)]

        eliminar_propiedades_nodo_por_filtro(uri, auth, nodo, filtros, propiedades)

    elif res5 == "6":
        nodo = input("Escribe el nombre del nodo al que le quieres eliminar propiedades: ")
        preg_res = input("¿Desea buscar por filtro? (Y/N) ")
        if preg_res == "Y":
            print("Agregue las propiedades y el valor para encontrar el nodo: ")
            filtros = {input("Propiedad: "): input("Valor: ")}

        else:
            filtros = {}
        
        n = int(input("¿Cuántas propiedades quiere eliminar?"))
        propiedades = [input("Propiedad: ") for i in range(n)]
    
    

In [99]:
def menu_crear_relaciones():
    nodo1 = input("Ingrese el nombre del primer nodo: ")
    res6 = input("¿Quiere filtrar este primer nodo? (Y/N) ").upper()
    nodo2 = input("Ingrese el nombre del segundo nodo: ")
    res7 = input("¿Quiere filtrar este segundo nodo? (Y/N) ").upper()
    tipo_relacion = input("Ingrese el tipo de relación que quiere crear (mayúsculas): ").upper()
    print("¿Cuántas propiedades quiere crear? (deben de ser al menos 3) ")
    n = int(input("¿Cuántas propiedades quiere crear?"))
    propiedades = {input("Propiedad: "): input("Valor: ") for i in range(n)}
    
    if res6 == "Y" and res7 == "Y":
        print("Ingrese la propiedad y el valor para filtrar el primer nodo")
        filtro1 = {input("Propiedad: "): input("Valor: ")}
        print("Ingrese la propiedad y el valor para filtrar el segundo nodo")
        filtro2 = {input("Propiedad: "): input("Valor: ")}
        crear_relacion_con_propiedades_por_filtro(uri, auth, nodo1, filtro1, nodo2, filtro2, tipo_relacion, propiedades)

    elif res6 == "Y" and res7 == "N":
        print("Ingrese la propiedad y el valor para filtrar el primer nodo")
        filtro1 = {input("Propiedad: "): input("Valor: ")}
        filtro2 = {}
        crear_relacion_con_propiedades_por_filtro(uri, auth, nodo1, filtro1, nodo2, filtro2, tipo_relacion, propiedades)

    elif res6 == "N" and res7 == "Y":
        filtro1 = {}
        print("Ingrese la propiedad y el valor para filtrar el segundo nodo")
        filtro2 = {input("Propiedad: "): input("Valor: ")}
        crear_relacion_con_propiedades_por_filtro(uri, auth, nodo1, filtro1, nodo2, filtro2, tipo_relacion, propiedades)

    elif res6 == "N" and res7 == "N":
        filtro1 = {}
        filtro2 = {}
        crear_relacion_con_propiedades_por_filtro(uri, auth, nodo1, filtro1, nodo2, filtro2, tipo_relacion, propiedades)
            
    else:
        print("No ingresaste una respuesta válida")

In [71]:
def menu_eliminacion_nodos():
    print("¿Qué desea realizar?")
    print("1. Eliminar un nodo")
    print("2. Eliminar más de un nodo")
    res8 = input("Ingrese el número de la opción: ")

    if res8 == "1":
        nodo = input("Ingrese el nombre del nodo que quiere eliminar")
        fil_preg = input("¿Quiere encontrar por filtro (Y/N)? ")
        if fil_preg == "Y":
            
            print("Ingrese el filtro para encontrar el nodo a eliminar")
            filtros = {input("Propiedad: "): input("Valor: ")}
    
            eliminar_un_nodo_por_filtro(uri, auth, nodo, filtros)
            
        else:
            filtros = {}
            eliminar_un_nodo_por_filtro(uri, auth, nodo, filtros)

    elif res8 == "2":
        n_nodos = int(input("¿Cuántos nodos desea eliminar? "))
        nodos = [input("Ingrese el nombre del nodo a eliminar: ") for i in range(n_nodos)]
        fil_preg = input("¿Quiere encontrar por filtro (Y/N)? ")
        if fil_preg == "Y":
            print("Ingrese el filtro para encontrar el nodo a eliminar")
            filtros = {input("Propiedad: "): input("Valor: ")}
            
            eliminar_multiples_nodos_por_labels_y_filtros(uri, auth, nodos, filtros)

        else:
            filtros = {}
            eliminar_multiples_nodos_por_labels_y_filtros(uri, auth, nodos, filtros)

In [88]:
def menu_eliminacion_relaciones():
    print("¿Qué desea realizar?")
    print("1. Eliminar una relación")
    print("2. Eliminar varias relaciones")
    res9 = input("Ingrese el número de la opción: ")

    if res9 == "1":
        nodo1 = input("Ingrese el nombre del primer nodo: ")
        preg_res = input("Quiere filtrar? (Y/N) ")
        if preg_res == "Y":
            print("Ingrese los datos para los filtros")
            filtro1 = {input("Propiedad: "): input("Valor: ")}
        else:
            filtro1 = {}
            
        nodo2 = input("Ingrese el nombre del segundo nodo: ")
        preg_res = input("Quiere filtrar? (Y/N) ")
        if preg_res == "Y":
            print("Ingrese los datos para el filtro del segundo nodo")
            filtro2 = {input("Propiedad: "): input("Valor: ")}

        else:
            filtro2 = {}
        tipo_relacion = input("Escribe el tipo de relacion que tiene: ").upper()

        eliminar_una_relacion_por_filtro(uri, auth, nodo1, filtro1, tipo_relacion, nodo2, filtro2)

    elif res9 == "2":

        nodo1 = input("Ingrese el nombre del primer nodo: ")
        preg_res = input("Quiere filtrar? (Y/N) ")
        if preg_res == "Y":
            print("Ingrese los datos para los filtros")
            filtro1 = {input("Propiedad: "): input("Valor: ")}
        else:
            filtro1 = {}
            
        nodo2 = input("Ingrese el nombre del segundo nodo: ")
        preg_res = input("Quiere filtrar? (Y/N) ")
        if preg_res == "Y":
            print("Ingrese los datos para el filtro del segundo nodo")
            filtro2 = {input("Propiedad: "): input("Valor: ")}

        else:
            filtro2 = {}
        tipo_relacion = input("Escribe el tipo de relacion que tiene: ").upper()

        eliminar_multiples_relaciones_por_filtro(uri, auth, nodo1, filtro1, tipo_relacion, nodo2, filtro2)

    

In [62]:
def main():
    exit = False
    while exit != True:
        primer_menu()
        res1 = input("Elija una opción (solo colocar número de opción): ")
        if res1 == "1":
            menu_nodos()

        elif res1 == "2":
            menu_consulta_nodos()

        elif res1 == "3":
            menu_operaciones_nodos()

        elif res1 == "4":
            menu_crear_relaciones()

        elif res1 == "5": #gestión de relacion
            pass #quitar el pass

        elif res1 == "6": #
            menu_eliminacion_nodos()

        elif res1 == "7":
            menu_eliminacion_relaciones()

        elif res1 == "8":
            exit = True

        else:
            print("Ingrese un número válido")
        

        

In [98]:
main()

¿Qué desea realizar?
1. Crear un nodo
2. Vizualizar nodos
3. Operaciones con nodos
4. Crear relaciones
5. Gestión de relaciones
6. Eliminar nodos
7. Eliminar relaciones
8. Salir


Elija una opción (solo colocar número de opción):  2
¿Desea buscar por label? (Y/N)  Y
¿Desea agregar filtros? (Y/N)  Y
¿Desea consultar un nodo o todos los nodos (uno/varios)?  varios
Ingrese el label que quiere buscar:  Actor
Propiedad:  Name
Valor:  Steve Martin


Resultados: [<Node element_id='4:06cb893f-12e2-4ae8-8349-8a2e0fcd7ad1:7' labels=frozenset({'Actor'}) properties={'Born_Date': 1953, 'ActorID': 24701, 'Gender': 2, 'Name': 'Steve Martin'}>]
¿Qué desea realizar?
1. Crear un nodo
2. Vizualizar nodos
3. Operaciones con nodos
4. Crear relaciones
5. Gestión de relaciones
6. Eliminar nodos
7. Eliminar relaciones
8. Salir


Elija una opción (solo colocar número de opción):  6


¿Qué desea realizar?
1. Eliminar un nodo
2. Eliminar más de un nodo


Ingrese el número de la opción:  2
¿Cuántos nodos desea eliminar?  2
Ingrese el nombre del nodo a eliminar:  Prueba1
Ingrese el nombre del nodo a eliminar:  Prueba2
¿Quiere encontrar por filtro (Y/N)?  N


¿Qué desea realizar?
1. Crear un nodo
2. Vizualizar nodos
3. Operaciones con nodos
4. Crear relaciones
5. Gestión de relaciones
6. Eliminar nodos
7. Eliminar relaciones
8. Salir


Elija una opción (solo colocar número de opción):  8


# Empieza lo anterior

In [13]:
def crear_relaciones(uri, auth, relationship_type, properties_keys):

    empty_properties = {key: "" if isinstance(key, str) else 0 for key in properties_keys} #cre

    query = f"""
    MATCH (a:Actor)-[r:{relationship_type}]->()
    SET r += $empty_properties
    RETURN COUNT(r) AS total_updated
    """

    with GraphDatabase.driver(uri, auth=auth) as driver:
        with driver.session() as session:
            result = session.run(query, empty_properties=empty_properties)
            return result.single()["total_updated"]

In [ ]:
crear_relaciones(uri, auth, )

In [4]:
def crear_nodo(uri, auth, label, properties_keys):

    empty_properties = {key: "" if isinstance(key, str) else 0 for key in properties_keys}

    query = f"""
    CREATE (n:{label} $empty_properties)
    RETURN n
    """

    with GraphDatabase.driver(uri, auth=auth) as driver:
        with driver.session() as session:
            result = session.run(query, empty_properties=empty_properties)
            return result.single()["n"]

In [12]:
propiedades = {"Name": "Franco"}
crear_nodo(uri, auth, "Actor", propiedades)

<Node element_id='4:e4d2ff4f-6a8b-4c6c-b02d-80359d5618b7:99430' labels=frozenset({'Actor'}) properties={'Name': ''}>

In [7]:
def crear_nodo_con_propiedades(uri, auth, label, properties):

    query = f"""
    CREATE (n:{label} $properties)
    RETURN n
    """

    with GraphDatabase.driver(uri, auth=auth) as driver:
        with driver.session() as session:
            result = session.run(query, properties=properties)
            return result.single()["n"]

In [10]:
crear_nodo_con_propiedades(uri, auth, "Actor", "Franco")

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Type mismatch for parameter 'properties': expected Map, Node or Relationship but was String (line 2, column 21 (offset: 21))
"    CREATE (n:Actor $properties)"
                     ^}

In [44]:
def ver_nodos(uri, auth, label):
    query = f"""
    MATCH (n:{label})
    RETURN n
    """

    with GraphDatabase.driver(uri, auth=auth) as driver:
        with driver.session() as session:
            result = session.run(query)
            return [record["n"] for record in result]

In [39]:
def borrar_un_nodo(uri, auth, node_id):

    query = """
    MATCH (n)
    WHERE ID(n) = $node_id
    DETACH DELETE n
    """

    with GraphDatabase.driver(uri, auth=auth) as driver:
        with driver.session() as session:
            session.run(query, node_id=node_id)


In [40]:
def borrar_varios_nodos(uri, auth, label, limit):

    query = f"""
    MATCH (n:{label})
    WITH n LIMIT $limit
    DETACH DELETE n
    """

    with GraphDatabase.driver(uri, auth=auth) as driver:
        with driver.session() as session:
            result = session.run(query, limit=limit)
            return result.consume().counters.nodes_deleted

In [41]:
def eliminar_relaciones(uri, auth, relationship_type):

    query = """
    MATCH ()-[r:{relationship_type}]->()
    DELETE r
    """

    with GraphDatabase.driver(uri, auth=auth) as driver:
        with driver.session() as session:
            result = session.run(query)
            return result.consume().counters.relationships_deleted

In [42]:
def agregar_propiedad(uri, auth, label, property_key, property_value):

    query = f"""
    MATCH (n:{label})
    SET n.{property_key} = $property_value
    RETURN COUNT(n) AS total_updated
    """

    with GraphDatabase.driver(uri, auth=auth) as driver:
        with driver.session() as session:
            result = session.run(query, property_value=property_value)
            return result.single()["total_updated"]

In [43]:
def quitar_propiedad(uri, auth, label, property_key):

    query = f"""
    MATCH (n:{label})
    REMOVE n.{property_key}
    RETURN COUNT(n) AS total_updated
    """

    with GraphDatabase.driver(uri, auth=auth) as driver:
        with driver.session() as session:
            result = session.run(query)
            return result.single()["total_updated"]

In [132]:
# Propiedades para ACTED_IN
properties_keys = ["ScreenTime", "CharacterImportance"]
relacion = "ACTED_IN"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 62841 relaciones ACTED_IN con nuevas propiedades vacías


In [25]:
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 62841 relaciones ACTED_IN con nuevas propiedades vacías


In [133]:
#propiedades para DIRECTED
properties_keys = ["Year", "FilmFestival"]
relacion = "DIRECTED"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 4669 relaciones DIRECTED con nuevas propiedades vacías


In [36]:
#Prpiedades para directed by
properties_keys = ["Year", "FilmFestival"]
relacion = "DIRECTED_BY"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 4669 relaciones DIRECTED_BY con nuevas propiedades vacías


In [135]:
#Prpiedades para FILMED_IN
properties_keys = ["City", "Days", "Year"]
relacion = "FILMED_IN"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 49350 relaciones FILMED_IN con nuevas propiedades vacías


In [ ]:
#Prpiedades para Likes
properties_keys = [""]
relacion = "LIKES"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

In [12]:
#Prpiedades para IS FRIENDS WITH
properties_keys = ["Since", "CommonInterests", "FrequenciaInteraccion"]
relacion = "IS_FRIENDS_WITH"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

NameError: name 'crear_relaciones' is not defined

In [137]:
#Prpiedades para IS TYPE
properties_keys = ["CommunityRating", "ReviewText", "MatchAccuracy"]
relacion = "IS_TYPE"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 91015 relaciones IS_TYPE con nuevas propiedades vacías


In [138]:
#Prpiedades para OPERATS IN
properties_keys = ["Marketshare",  "YearsEstablished"]
relacion = "OPERATES_IN"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 34853 relaciones OPERATES_IN con nuevas propiedades vacías


In [139]:
#Prpiedades para PRODUCED BY
properties_keys = ["Budget", "BoxOffice", "Reception"]
relacion = "PRODUCED_BY"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 68434 relaciones PRODUCED_BY con nuevas propiedades vacías


In [140]:
#Prpiedades para RATED
properties_keys = [ "RatingDate", "ReviewText"]
relacion = "RATED"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 2514 relaciones RATED con nuevas propiedades vacías


In [141]:
#Prpiedades para RECOMMENDED_BY
properties_keys = ["Ratingvalue", "RatingDate", "ReviewText"]
relacion = "WAS_RECOMMENDED_BY"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

Se actualizaron 2514 relaciones WAS_RECOMMENDED_BY con nuevas propiedades vacías


In [ ]:
#Prpiedades para RECOMMENDED_BY
properties_keys = ["Ratingvalue", "RatingDate", "ReviewText"]
relacion = "WAS_RECOMMENDED_BY"
total = crear_relaciones(uri, auth, relacion, properties_keys)
print(f"Se actualizaron {total} relaciones {relacion} con nuevas propiedades vacías")

## Recomendador

In [3]:
driver = GraphDatabase.driver(uri, auth=auth)

In [13]:
#verificar que exista el usuario
def user_exists(user_id):
    """
    Retorna True si existe un nodo User con la propiedad user_id = user_id,
    False en caso contrario.
    """
    query = """
    MATCH (u:User {UserID: $user_id})
    RETURN count(u) AS count
    """
    with driver.session() as session:
        result = session.run(query, user_id=user_id)
        record = result.single()
        if record and record["count"] > 0:
            return True
        return False

In [21]:
def recommend_by_user(user_id, rating_threshold=3.5, top_n=10):
    """
    Recomienda películas que el usuario no haya visto, pero que sus amigos
    hayan valorado por encima de rating_threshold.
    """
    # Primero, verificamos si el usuario existe
    if not user_exists(user_id):
        print(f"Error: No se encontró ningún usuario con ID = {user_id}.")
        return []
    
    query = """
    MATCH (u:User {UserID: $user_id})-[r:RATED]->(m:Movie)
    WITH m, avg(r.Rating) AS user_avg_rating
    WHERE user_avg_rating >= $rating_threshold
    RETURN m.Title AS title, user_avg_rating
    ORDER BY user_avg_rating DESC
    LIMIT $top_n;
    """
    with driver.session() as session:
        result = session.run(
            query,
            user_id=user_id,
            rating_threshold=rating_threshold,
            top_n=top_n
        )
        return [record.data() for record in result]


In [15]:
def search_movies(
    genre=None,
    year=None,
    duration=None,
    keywords=None,
    company=None,
    director=None,
    top_n=10
):
    
    base_query = """
    MATCH (m:Movie)
    OPTIONAL MATCH (m)-[:IS_TYPE]->(g:Genre)
    OPTIONAL MATCH (m)-[:PRODUCED_BY]->(c:Company)
    OPTIONAL MATCH (m)<-[:DIRECTED]-(d:Director)
    WHERE 1=1
    """
    conditions = []
    
    if genre:
        conditions.append("g.Name = $genre")
    
    if year:
        conditions.append("(date(m.Released).year >= $yearMin AND date(m.Released).year <= $yearMax)")
    
    if duration:
        conditions.append("(m.Runtime >= $durationMin AND m.Runtime <= $durationMax)")
    
    if keywords:
        conditions.append("m.Keyword CONTAINS $keywords")
    
    if company:
        conditions.append("c.Name CONTAINS $company")
    
    if director:
        conditions.append("d.Name CONTAINS $director")
    
    if conditions:
        base_query += " OR " + " OR ".join(conditions)
    
    base_query += """
    RETURN DISTINCT m.Title AS title, 
        date(m.Released).year AS year, 
        m.Runtime AS duration
    ORDER BY date(m.Released).year DESC, m.Runtime DESC, m.Title
    LIMIT $top_n
    """
    
    params = {
        "genre": genre,
        "keywords": keywords,
        "company": company,
        "director": director,
        "top_n": top_n,
        "yearMin": year - 2 if year else None,
        "yearMax": year + 2 if year else None,
        "durationMin": duration - 30 if duration else None,
        "durationMax": duration + 30 if duration else None
    }
    
    with driver.session() as session:
        result = session.run(base_query, **params)
        return [record.data() for record in result]


In [24]:
from datetime import datetime

def main():
    print("Bienvenido al sistema de recomendación de películas.")
    print("1) Recomendación por usuario (basada en amigos).")
    print("2) Búsqueda personalizada (género, año ± 2, duración ± 30, palabras clave, compañía, director).")
    
    opcion = input("Elige una opción (1/2): ").strip()
    
    if opcion == "1":
        # Recomendación por usuario
        user_id_str = input("Ingresa el ID del usuario: ").strip()
        if not user_id_str.isdigit():
            print("Error: El ID de usuario debe ser un número entero.")
            return
        user_id = int(user_id_str)
        
        # Se recomienda con threshold de rating y top_n fijos, puedes modificar si quieres
        recomendaciones = recommend_by_user(user_id, rating_threshold=3.5, top_n=10)
        
        if recomendaciones:
            print(f"\n=== Recomendaciones para el usuario {user_id} (rating >= 3.5) ===")
            for rec in recomendaciones:
                print(f"{rec['title']} (Valoración promedio: {rec['user_avg_rating']:.2f})")
        else:
            print("No se encontraron recomendaciones para este usuario o el usuario no existe.")
    
    elif opcion == "2":
        # Búsqueda personalizada
        print("Puedes dejar cualquier campo en blanco para omitirlo.")
        
        genre = input("Género: ").strip()
        year_input = input("Año (± 2): ").strip()
        duration_input = input("Duración aproximada (minutos ± 30): ").strip()
        keywords = input("Palabras clave (en la columna 'Keywords'): ").strip()
        company = input("Compañía/Estudio (búsqueda parcial): ").strip()
        director = input("Director (búsqueda parcial): ").strip()
        
        # Validamos si year y duration son dígitos
        year = None
        if year_input:
            if year_input.isdigit():
                year = int(year_input)
            else:
                print("Aviso: El año ingresado no es un número. Se ignorará el filtro de año.")
        
        duration = None
        if duration_input:
            if duration_input.isdigit():
                duration = int(duration_input)
            else:
                print("Aviso: La duración ingresada no es un número. Se ignorará el filtro de duración.")
        
        resultados = search_movies(
            genre=genre if genre else None,
            year=year,
            duration=duration,
            keywords=keywords if keywords else None,
            company=company if company else None,
            director=director if director else None,
            top_n=10
        )
        
        if resultados:
            print("\n=== Resultados de búsqueda ===")
            for r in resultados:
                print(f"{r['title']} (Año: {r['year']}, Duración: {r['duration']} min)")
        else:
            print("No se encontraron películas que cumplan los criterios (o no existen en la BD).")
    
    else:
        print("Opción no válida.")


In [26]:
if __name__ == "__main__":
    try:
        driver = GraphDatabase.driver(uri, auth=auth)
        main()
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    finally:
        driver.close()

Bienvenido al sistema de recomendación de películas.
1) Recomendación por usuario (basada en amigos).
2) Búsqueda personalizada (género, año ± 2, duración ± 30, palabras clave, compañía, director).

=== Recomendaciones para el usuario 2 (rating >= 3.5) ===
Kicking Off (Valoración promedio: 4.80)
Cose da pazzi (Valoración promedio: 4.78)
Kite Liberator (Valoración promedio: 4.72)
My Awkward Sexual Adventure (Valoración promedio: 4.68)
Wittgenstein (Valoración promedio: 4.44)
Who Saw Him Die? (Valoración promedio: 4.43)
Fantozzi The Return (Valoración promedio: 4.38)
Amen (Valoración promedio: 4.36)
The Man Who Left His Will on Film (Valoración promedio: 4.31)
Time Regained (Valoración promedio: 3.83)


In [31]:
import datetime
def menu():
    print("BIENVENIDO AL RECOMENDADOR DE PELICULAS")
    print("Ingresa tus gustos y preferencias a continuación:")


def validar_fecha(fecha_texto):
    try:
        datetime.datetime.strptime(fecha_texto, "%Y-%m-%d")
        return True
    except ValueError:
        return False

def procesar_entrada_lista(entrada, capitalizar=False):
    lista = entrada.split(',')
    if capitalizar:
        # Capitalizar la primera letra de cada palabra en cada elemento de la lista.
        return [item.strip().title() for item in lista]
    else:
        return [item.strip() for item in lista]

def variables_recomendacion():
    genero = procesar_entrada_lista(input("Ingresa los géneros que te gustan, separados por comas: "), True)
    
    while True:
        released = input("Ingresa la fecha de estreno que prefieres (formato YYYY-MM-DD): ").strip()
        if validar_fecha(released):
            break
        print("Formato de fecha incorrecto, por favor intenta de nuevo.")
    
    while True:
        duracion = input("Ingresa la duración mínima que prefieres para la película (en minutos): ").strip()
        if duracion.isdigit():
            duracion = int(duracion)
            break
        print("Por favor ingresa un número válido.")
    
    keywords = procesar_entrada_lista(input("Ingresa palabras clave relacionadas con la película, separadas por comas: "))
    studio = input("Ingresa el estudio de cine que prefieres: ").strip().title()
    director = input("Ingresa el nombre del director que te gustaría ver: ").strip().title()
    actors = procesar_entrada_lista(input("Ingresa los nombres de los actores que te gustan, separados por comas: "), True)

    return genero, released, duracion, keywords, studio, director, actors


#############################################

def neo4j_connect(uri, auth):
    driver = GraphDatabase.driver(uri, auth=auth)
    return driver

def get_movie_recommendations(driver, genero, released, duracion, keywords, studio, director, actors):
    # Ajustar el rango de fecha
    year = int(released.split("-")[0])
    min_year = year - 5
    max_year = year + 5
    
    # Ajustar el rango de duración
    min_runtime = duracion - 30
    max_runtime = duracion + 30
    
    query = """
    MATCH (m:Movie)<-[:ACTED_IN]-(a:Actor), 
        (m)-[:IS_TYPE]->(g:Genre), 
        (m)<-[:DIRECTED_BY]-(d:Director), 
        (m)<-[:PRODUCED_BY]-(c:Company)
    WITH m, 
        collect(a.Name) AS actorNames,
        count(DISTINCT g.Name) AS genreMatch,
        CASE WHEN m.Released >= $minYear AND m.Released <= $maxYear THEN 1 ELSE 0 END AS yearMatch,
        CASE WHEN m.Runtime >= $minRuntime AND m.Runtime <= $maxRuntime THEN 1 ELSE 0 END AS runtimeMatch,
        CASE WHEN any(keyword IN m.Keyword WHERE keyword IN $keywords) THEN 1 ELSE 0 END AS keywordMatch,
        CASE WHEN c.Name = $studio THEN 1 ELSE 0 END AS studioMatch,
        CASE WHEN d.Name = $director THEN 1 ELSE 0 END AS directorMatch
    WHERE any(actor IN $actors WHERE actor IN actorNames)
    WITH m, 
        genreMatch + yearMatch + runtimeMatch + keywordMatch + studioMatch + directorMatch AS matchScore
    WHERE matchScore >= 2  // Solo tomar películas que cumplan al menos 3 condiciones
    RETURN m.Title AS Title, m.Released AS Released, m.Runtime AS Runtime, matchScore
    ORDER BY matchScore DESC
    LIMIT 10;

    """

    
    with driver.session() as session:
        result = session.run(query, {
            'genres': genero,
            'minYear': min_year,
            'maxYear': max_year,
            'minRuntime': min_runtime,
            'maxRuntime': max_runtime,
            'keywords': keywords,
            'studio': studio,
            'director': director,
            'actors': actors
        })
        return list(result)

"""
# Conexión a Neo4j

driver = neo4j_connect(uri, auth)

################### MAIN

menu()

# Recoger inputs del usuario
inputs = variables_recomendacion()
genero, released, duracion, keywords, studio, director, actors = inputs

# Obtener recomendaciones
recommendations = get_movie_recommendations(driver, *inputs)

#Imprimir recomendaciones
for movie in recommendations:
    print(movie['Title'], movie['Released'], movie['Runtime'])
############################################

"""

"\n# Conexión a Neo4j\n\ndriver = neo4j_connect(uri, auth)\n\n################### MAIN\n\nmenu()\n\n# Recoger inputs del usuario\ninputs = variables_recomendacion()\ngenero, released, duracion, keywords, studio, director, actors = inputs\n\n# Obtener recomendaciones\nrecommendations = get_movie_recommendations(driver, *inputs)\n\n#Imprimir recomendaciones\nfor movie in recommendations:\n    print(movie['Title'], movie['Released'], movie['Runtime'])\n############################################\n\n"

In [32]:
if __name__ == "__main__":
    try:
        driver = GraphDatabase.driver(uri, auth=auth)
        main()
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    finally:
        driver.close()

Bienvenido al sistema de recomendación de películas.
1) Recomendación por usuario (basada en amigos).
2) Búsqueda personalizada (género, año ± 2, duración ± 30, palabras clave, compañía, director).
Puedes dejar cualquier campo en blanco para omitirlo.

=== Resultados de búsqueda ===
Allende en su laberinto (Año: None, Duración: None min)
Aprel (Año: None, Duración: None min)
Aurinkotuuli (Año: None, Duración: None min)
Bad Dad Rehab (Año: None, Duración: None min)
Dolpo Tulku - Heimkehr in den Himalaya (Año: None, Duración: None min)
Normaal: Ik Kom Altied Weer Terug (Año: None, Duración: None min)
Pajęczarki (Año: None, Duración: None min)
Pölynimurikauppiaat (Año: None, Duración: None min)
Supermassive Black Holes (Año: None, Duración: None min)
Vous êtes très jolie, mademoiselle (Año: None, Duración: None min)


In [35]:
import datetime
from neo4j import GraphDatabase

def menu():
    print("BIENVENIDO AL RECOMENDADOR DE PELICULAS")
    print("Ingresa tus gustos y preferencias a continuación:")

def validar_fecha(fecha_texto):
    try:
        datetime.datetime.strptime(fecha_texto, "%Y-%m-%d")
        return True
    except ValueError:
        return False

def procesar_entrada_lista(entrada, capitalizar=False):
    lista = entrada.split(',')
    if capitalizar:
        return [item.strip().title() for item in lista]
    else:
        return [item.strip() for item in lista]

def variables_recomendacion():
    genero = procesar_entrada_lista(input("Ingresa los géneros que te gustan, separados por comas: "), True)
    while True:
        released = input("Ingresa la fecha de estreno que prefieres (formato YYYY-MM-DD): ").strip()
        if validar_fecha(released):
            break
        print("Formato de fecha incorrecto, por favor intenta de nuevo.")
    while True:
        duracion = input("Ingresa la duración mínima que prefieres para la película (en minutos): ").strip()
        if duracion.isdigit():
            duracion = int(duracion)
            break
        print("Por favor ingresa un número válido.")
    keywords = procesar_entrada_lista(input("Ingresa palabras clave relacionadas con la película, separadas por comas: "))
    studio = input("Ingresa el estudio de cine que prefieres: ").strip().title()
    director = input("Ingresa el nombre del director que te gustaría ver: ").strip().title()
    actors = procesar_entrada_lista(input("Ingresa los nombres de los actores que te gustan, separados por comas: "), True)
    return genero, released, duracion, keywords, studio, director, actors

def neo4j_connect(uri, user, password):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    return driver

uri = "neo4j+ssc://4d9cb826.databases.neo4j.io"
user, password = ("neo4j", "_4X58M73ITpFpJVDUxHA2K1Bu92nWAUvuvpz4NzsFLU")
driver = neo4j_connect(uri, user, password)

try:
    driver.verify_connectivity()
    print("Conexión exitosa.")
except Exception as e:
    print(f"Error al conectar: {e}")

def get_movie_recommendations(driver, genero, released, duracion, keywords, studio, director, actors):
    # Convertir la fecha ingresada al año y crear rangos de años
    year = datetime.datetime.strptime(released, "%Y-%m-%d").year
    min_year = f"{year-10}-01-01"
    max_year = f"{year+10}-12-31"
    
    # Ajustar el rango de duración
    min_runtime = duracion - 30
    max_runtime = duracion + 30
    
    # Preparar la consulta Cypher
    query = """
    MATCH (m:Movie)
    OPTIONAL MATCH (m)-[:IS_TYPE]->(g:Genre)
    WITH m, COLLECT(g.Name) AS Genres
    OPTIONAL MATCH (m)<-[:DIRECTED_BY]-(d:Director)
    WITH m, Genres, COLLECT(d.Name) AS Directors
    OPTIONAL MATCH (m)<-[:PRODUCED_BY]-(c:Company)
    WITH m, Genres, Directors, COLLECT(c.Name) AS Companies
    OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Actor)
    WITH m, Genres, Directors, Companies, COLLECT(a.Name) AS Actors
    WITH m, Genres, Directors, Companies, Actors,
        ((CASE WHEN ANY(genre IN Genres WHERE genre IN $genres) THEN 1 ELSE 0 END) +
        (CASE WHEN $director IS NOT NULL AND ANY(director IN Directors WHERE director = $director) THEN 1 ELSE 0 END) +
        (CASE WHEN $studio IS NOT NULL AND ANY(company IN Companies WHERE company = $studio) THEN 1 ELSE 0 END) +
        (CASE WHEN ANY(actor IN Actors WHERE actor IN $actors) THEN 1 ELSE 0 END) +
        (CASE WHEN ANY(keyword IN split(m.Keyword, ',') WHERE keyword IN $keywords) THEN 1 ELSE 0 END)
        ) AS Score
    RETURN m AS Movie, Score, Genres, Directors, Companies, Actors
    ORDER BY Score DESC
    LIMIT 10

    """
    
    # Ejecutar la consulta
    with driver.session() as session:
        result = session.run(query, {
            'minYear': min_year,
            'maxYear': max_year,
            'minRuntime': min_runtime,
            'maxRuntime': max_runtime,
            'genres': genero,
            'keywords': keywords,
            'studio': studio,
            'director': director,
            'actors': actors
        })
        return list(result)




menu()
inputs = variables_recomendacion()
genero, released, duracion, keywords, studio, director, actors = inputs
recommendations = get_movie_recommendations(driver, *inputs)

for movie in recommendations:
    movie_info = movie['Movie']
    genres = ', '.join(movie['Genres'])  # 'Genres' es una lista de nombres de géneros
    directors = ', '.join(movie['Directors'])  # Similar para 'Directors'
    companies = ', '.join(movie['Companies'])  # Y 'Companies'
    actors = ', '.join(movie['Actors'])  # Y 'Actors'
    score = movie['Score']  # Acceder al puntaje calculado


    print(f"Title: {movie_info['Title']}")
    print(f"Released: {movie_info['Released']}")
    print(f"Runtime: {movie_info['Runtime']}")
    print(f"Genres: {genres}")
    print(f"Directors: {directors}")
    print(f"Companies: {companies}")
    print(f"Actors: {actors}")
    print(f"Score: {score}")  # Imprimir el puntaje de coincidencia
    print("--------------------------------------------------")


Conexión exitosa.
BIENVENIDO AL RECOMENDADOR DE PELICULAS
Ingresa tus gustos y preferencias a continuación:


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "\n    MATCH (m:Movie)\n    OPTIONAL MATCH (m)-[:IS_TYPE]->(g:Genre)\n    WITH m, COLLECT(g.Name) AS Genres\n    OPTIONAL MATCH (m)<-[:DIRECTED_BY]-(d:Director)\n    WITH m, Genres, COLLECT(d.Name) AS Directors\n    OPTIONAL MATCH (m)<-[:PRODUCED_BY]-(c:Company)\n    WITH m, Genres, Directors, COLLECT(c.Name) AS Companies\n    OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Actor)\n    WITH m, Genres, Directors, Companies, COLLECT(a.Name) AS Actors\n    WITH m, Genres, Directors, Companies, Actors,\n        ((CASE WHEN ANY(genre IN Genres WHERE genre IN $genres) THEN 1 ELSE 0 END) +\n        (CASE WHEN $director IS NOT NULL AND ANY(director IN Directors WHERE director = $director) THEN 1 ELSE 0 EN

Title: The Break-Up
Released: 2006-06-01T00:00:00.000000000+00:00
Runtime: 106.0
Genres: Comedy, Romance
Directors: 
Companies: 
Actors: 
Score: 2
--------------------------------------------------
Title: Rio
Released: 2011-04-03T00:00:00.000000000+00:00
Runtime: 96.0
Genres: Animation, Comedy, Family, Adventure
Directors: 
Companies: 
Actors: Leslie Mann, Jamie Foxx, Tracy Morgan, Jesse Eisenberg, Rodrigo Santoro, Carlos Ponce, Anne Hathaway, Wanda Sykes, George Lopez, Jake T. Austin, Jane Lynch, Will.i.am, Jemaine Clement
Score: 2
--------------------------------------------------
Title: Road House
Released: 1948-09-22T00:00:00.000000000+00:00
Runtime: 95.0
Genres: Romance, Drama, Action, Thriller
Directors: 
Companies: 
Actors: 
Score: 2
--------------------------------------------------
Title: Hoodwinked!
Released: 2005-12-16T00:00:00.000000000+00:00
Runtime: 80.0
Genres: Animation, Comedy, Family
Directors: 
Companies: 
Actors: Glenn Close, David Ogden Stiers, Chazz Palminteri, Ji

In [34]:
if __name__ == "__main__":
    try:
        driver = GraphDatabase.driver(uri, auth=auth)
        main()
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    finally:
        driver.close()

Bienvenido al sistema de recomendación de películas.
1) Recomendación por usuario (basada en amigos).
2) Búsqueda personalizada (género, año ± 2, duración ± 30, palabras clave, compañía, director).
Opción no válida.
